In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
import os
import pandas as pd
import numpy as np
from functools import reduce


macro_folder = '/content/drive/MyDrive/DS320_GAS_PRICE_FUSION/Data/Raw Data/MacroEcon'
gas_price_folder   = "/content/drive/MyDrive/DS320_GAS_PRICE_FUSION/Data/Raw Data/Target Data"

In [20]:
# -------------------------------------------------------
# Helper function for monthly CSVs (2 columns: date + value)
# -------------------------------------------------------
def load_monthly_csv(path, value_name):
    df = pd.read_csv(path)
    df.columns = ["date", value_name]
    df["date"] = pd.to_datetime(df["date"])
    # Group by MONTH, output date = 1st of each month
    df = df.groupby(pd.Grouper(key="date", freq="MS")).mean().reset_index()
    return df


# -------------------------------------------------------
# Load the 3 monthly CSVs in MacroEcon
# -------------------------------------------------------
fed_funds = load_monthly_csv(
    os.path.join(macro_folder, "fed_funds_rate_monthly.csv"),
    "fed_funds_rate"
)

interest = load_monthly_csv(
    os.path.join(macro_folder, "interest_rates_monthly.csv"),
    "interest_rate"
)

dollar_index = load_monthly_csv(
    os.path.join(macro_folder, "nominal_dollar_index_monthly.csv"),
    "dollar_index"
)

# -------------------------------------------------------
# Helper to load Excel files with: Year column + month columns
# -------------------------------------------------------
def load_year_month_excel(path, value_name, header_row):
    # Load with header at specific row
    df = pd.read_excel(path, skiprows=header_row)

    # Clean column labels (remove dots)
    df.columns = [str(c).strip().replace(".", "") for c in df.columns]

    # Drop blank rows
    df = df.dropna(subset=["Year"])

    # Convert Year floats → ints
    df["Year"] = df["Year"].astype(int)

    # Month → number mapping
    month_map = {
        'Jan':'01','Feb':'02','Mar':'03','Apr':'04','May':'05','Jun':'06',
        'Jul':'07','Aug':'08','Sep':'09','Oct':'10','Nov':'11','Dec':'12'
    }

    # Melt wide → long
    df_long = df.melt(id_vars="Year", var_name="Month", value_name=value_name)
    df_long = df_long[df_long["Month"].isin(month_map.keys())]

    # Construct date
    df_long["date"] = pd.to_datetime(
        df_long["Year"].astype(str) + "-" + df_long["Month"].map(month_map) + "-01",
        format="%Y-%m-%d"
    )

    return df_long[["date", value_name]].sort_values("date").reset_index(drop=True)


# -------------------------------------------------------
# Load CPI and Unemployment (with correct header rows)
# -------------------------------------------------------
cpi = load_year_month_excel(
    os.path.join(macro_folder, "cpi_monthly.xlsx"),
    "cpi",
    header_row=3   # header in Excel row 4
)

unemp = load_year_month_excel(
    os.path.join(macro_folder, "unemployment_monthly.xlsx"),
    "unemployment_rate",
    header_row=11  # header in Excel row 12
)

# -------------------------------------------------------
# Load weekly gas price → convert to monthly
# -------------------------------------------------------
gas = pd.read_csv(os.path.join(gas_price_folder, "eia_prices_weekly.csv"))

# Keep only date column + price column
gas = gas.iloc[:, [0, 2]]
gas.columns = ["date", "gas_price"]

gas["date"] = pd.to_datetime(gas["date"])
gas = gas.groupby(pd.Grouper(key="date", freq="MS")).mean().reset_index()

# -------------------------------------------------------
# Merge ALL macro data together
# -------------------------------------------------------
dfs = [fed_funds, interest, dollar_index, cpi, unemp, gas]

macro = reduce(lambda left, right: pd.merge(left, right, on="date", how="outer"), dfs)
macro = macro.sort_values("date").reset_index(drop=True)

macro.head(), macro.tail()

/usr/local/lib/python3.12/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


(        date  fed_funds_rate  interest_rate  dollar_index  cpi  \
 0 1913-01-01             NaN            NaN           NaN  9.8   
 1 1913-02-01             NaN            NaN           NaN  9.8   
 2 1913-03-01             NaN            NaN           NaN  9.8   
 3 1913-04-01             NaN            NaN           NaN  9.8   
 4 1913-05-01             NaN            NaN           NaN  9.7   
 
    unemployment_rate  gas_price  
 0                NaN        NaN  
 1                NaN        NaN  
 2                NaN        NaN  
 3                NaN        NaN  
 4                NaN        NaN  ,
            date  fed_funds_rate  interest_rate  dollar_index      cpi  \
 1351 2025-08-01            4.33       4.264762    120.984448  323.976   
 1352 2025-09-01            4.22       4.120476    120.453400    324.8   
 1353 2025-10-01            4.09       4.061818    121.171182        –   
 1354 2025-11-01            3.88       4.093889    121.803756        –   
 1355 2025-12-0

In [22]:
macro.to_csv("/content/drive/MyDrive/DS320_GAS_PRICE_FUSION/Data/Raw Data/MacroEcon/macro_table.csv", index=False)